# Imports

In [1]:
using LinearAlgebra
using SparseArrays
using Distributions
using PyCall
using Statistics
using BenchmarkTools

@pyimport numpy 

# GMRES

In [2]:
function gmres(A, B, X0, max_iter)
    r = B - A * X0
    x = NaN
    q = Array{Any}(zeros(max_iter)) #array any, gdyż przechowuje i wektory i pojedyncze liczby 
    q[1] = r/norm(r)
    h = zeros((max_iter + 1, max_iter))
    
    #for m=1:max_iter
    for m=1:min(max_iter, size(A, 1))
        y = A * q[m]
        
        #for j=1:m-1
        for j=1:m
            h[j, m] = (q[j]' * y)[1]
            y = y - h[j, m] * q[j] 
        end
        
        h[m + 1, m] = norm(y)
        
        if (h[m + 1, m] != 0) && (m != max_iter)
            q[m + 1] = y / h[m + 1, m]
        end

        b = zeros(max_iter + 1)
        b[1] = norm(r)
        result = (h \ b)

        if m > max_iter - 2
            z = reduce(hcat, q[:])
        else
            z = q
        end
        
        x = numpy.dot(z, result)+X0
        
    end
    
    return x
end

gmres (generic function with 1 method)

# Restarted GMRES

In [3]:
function restarted_gmres(A, B, X0, max_iter, res)
    new_x0 = gmres(A, B, X0, res)
    for i=2*res:res:max_iter
        new_x0 = gmres(A, B, new_x0, res)
    end
    return new_x0
end

restarted_gmres (generic function with 1 method)

# BICGSTAB

In [11]:
function bicgstab(A, B, init_guess, iterations, accuracy)
    x_prev = init_guess
    r_prev = B - A*x_prev
    r0_arb = r_prev
    rho_prev = alpha = omega_prev = 1.0
    v_prev = p_prev = rho = 0.0
    result = NaN
    for iter = 1:iterations
        rho = dot(r0_arb, r_prev)
        beta = (rho/rho_prev) * (alpha/omega_prev)
        p = r_prev .+ (beta* (p_prev - omega_prev * v_prev))
        v = A * p
        alpha = rho'/(dot(r0_arb, v))
        h = x_prev + alpha*p
        s = r_prev - alpha*v
        if(norm(s) < accuracy)
            result = h
            break
        end
        t = A * s
        omega = dot(t, s) / dot(t, t)
        x = h + omega*s
        r = s - omega*t
        if(norm(r) < accuracy || iter == iterations)
            result = x
            break
        end
        r_prev = r
        x_prev = x
        p_prev = p
        v_prev = v
        omega_prev = omega
        rho_prev = rho
    end
    return result
end

bicgstab (generic function with 1 method)

In [5]:
#generacja macierzy A oraz wektorów B i X0
sizee = 200
A = Symmetric(sprand(sizee,sizee,0.5))
B = rand(Uniform(0,1), sizee, 1)
X0 = rand(Uniform(0,1), sizee, 1)

200×1 Matrix{Float64}:
 0.7847887768131281
 0.8302573038249981
 0.1940030487189599
 0.998688686473242
 0.2720887121865525
 0.3186591139782702
 0.8062995405074715
 0.17754942955472486
 0.03841199474028634
 0.4120192376172096
 0.2684315614071884
 0.5873698251693249
 0.24291303760374716
 ⋮
 0.17324965519832647
 0.27087110869735653
 0.4496249904692653
 0.6435854092925153
 0.02639690016853491
 0.7072451920279086
 0.4716496109209558
 0.04305849867958167
 0.3089653992187136
 0.9004617639055681
 0.9470470374110058
 0.12514512605754535

In [6]:
#gmres
max_iter = 500
res = 150

#bicgstab
accuracy = 1e-10

1.0e-10

In [7]:
x3 = A\B

200×1 Matrix{Float64}:
 -0.39245624969643517
 -0.0874632637470291
  0.31297419729249576
  1.2223080519414389
  0.060686675617901246
  0.6034493759137114
 -0.1696955964225211
  0.0732531040955664
  0.2052349310122986
 -0.33915178657939665
 -0.42268980053968147
  0.1422193374314493
  0.3613285624580537
  ⋮
  1.419490345142858
 -0.38582957232255205
 -0.08561411847323368
  0.04553962226053972
  0.28534300913841404
  0.15250014708425827
  0.8437471034485475
  0.2353880176281848
  0.3131646174078803
  0.39130486062795816
  0.2581513818770215
 -0.1136130333004839

In [8]:
x2 = restarted_gmres(A, B, X0, 10*max_iter, res)
#x2 = gmres(A, B, X0, max_iter, accuracy, x3)

200×1 Matrix{Float64}:
 -0.39245624973945903
 -0.08746326380470033
  0.31297419717275476
  1.2223080517210587
  0.06068667563054799
  0.6034493758200269
 -0.1696955964245243
  0.07325310408465323
  0.2052349310294478
 -0.3391517864718118
 -0.42268980048941707
  0.14221933751151544
  0.36132856242543737
  ⋮
  1.4194903449769911
 -0.38582957231814646
 -0.08561411847505594
  0.045539622249904384
  0.2853430089992492
  0.15250014701010786
  0.8437471032534616
  0.23538801757302424
  0.3131646173497909
  0.391304860649726
  0.25815138181469227
 -0.11361303326052084

In [9]:
xd = gmres(A, B, X0, max_iter)

200×1 Matrix{Float64}:
 -0.39245624969642745
 -0.08746326374701086
  0.3129741972924876
  1.222308051941406
  0.06068667561786961
  0.6034493759137154
 -0.16969559642252952
  0.07325310409556293
  0.20523493101230988
 -0.33915178657940137
 -0.42268980053964544
  0.14221933743145165
  0.3613285624580641
  ⋮
  1.4194903451428418
 -0.3858295723225451
 -0.08561411847323597
  0.04553962226056729
  0.2853430091383826
  0.1525001470842694
  0.8437471034485379
  0.235388017628167
  0.3131646174078442
  0.3913048606279643
  0.25815138187699405
 -0.11361303330046141

In [12]:
x1 = bicgstab(A, B, X0, max_iter, accuracy)

200×1 Matrix{Float64}:
 -0.39243004510513024
 -0.08750884423750908
  0.31289425487590383
  1.2223353662136163
  0.06075251650420016
  0.6034147319536532
 -0.16966880336821413
  0.073272478010443
  0.2052265036431813
 -0.33912746526631526
 -0.4227271774406036
  0.1422223451237996
  0.36130977380060286
  ⋮
  1.4194592268946384
 -0.3858456681909332
 -0.08563458674472453
  0.04545836452530903
  0.28536959989651806
  0.1524818976257694
  0.8437007606991952
  0.23542574090642693
  0.3132197145678225
  0.39129513288769363
  0.2581595544803807
 -0.11364437971373698

In [127]:
#absolute error
abs_res = broadcast(abs, x2-x3)
abs_normal = broadcast(abs, xd-x3)

mean_abs_res = mean(abs_res)
mean_abs_normal = mean(abs_normal)

println("restarted:")
println(mean_abs_res)
println("base:")
println(mean_abs_normal)

restarted:
2.8495706228676145e-15
base:
1.6406135347826757e-14


In [128]:
#relative error (percentage)
rel_res = abs_res./x2 * 100
rel_normal = abs_normal./xd * 100

mean_rel_res = abs(mean(rel_res))
mean_rel_normal = abs(mean(rel_normal))

println("restarted:")
println(mean_rel_res)
println("base:")
println(mean_rel_normal)

restarted:
4.678287332097696e-13
base:
2.0531274930885808e-12
